<a href="https://colab.research.google.com/github/IvanGonzal0/Big-Data-y-ML/blob/main/Laboratorio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd

In [ ]:
path = '/content/drive/MyDrive/UPSO ML & BD/Dataset de ventas2'

archivos = os.listdir(path)
archivos

['Dataset_de_ventas_Abril.csv',
 'Dataset_de_ventas_Agosto.csv',
 'Dataset_de_ventas_Diciembre.csv',
 'Dataset_de_ventas_Enero.csv',
 'Dataset_de_ventas_Febrero.csv',
 'Dataset_de_ventas_Julio.csv',
 'Dataset_de_ventas_Junio.csv',
 'Dataset_de_ventas_Marzo.csv',
 'Dataset_de_ventas_Mayo.csv',
 'Dataset_de_ventas_Noviembre.csv',
 'Dataset_de_ventas_Octubre.csv',
 'Dataset_de_ventas_Septiembre.csv',
 '.ipynb_checkpoints']

In [ ]:
# Ahora voy a crear un diccionario para guardar estos archivos, para luego hacer un df

dataframes = {}

for archivo in archivos:
  if archivo.endswith('csv'):
    mes = archivo.split('_')[3].split('.')[0] # obtener el nombre del mes solo
    #aca obtenemos la ruta completa de donde se encuentra el csv de cada mes
    path_completo = os.path.join(path, archivo) # obtener ruta de acceso completa
    df = pd.read_csv(path_completo)   #cargar el archivo csv en un df
    #vamos a trabajar sobre la copia del df original
    df_ventas = df.copy()


    # SANITIZAR CADA DF ANTES DE CARGARLO
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors = 'coerce') #errors = coerce transfora lo
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors = 'coerce')
    #esta es una mascara que me genera un true si hay un nan, y sino hay nan me pone un false
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]
    #aca elimino los nan's
    df_ventas = df_ventas.dropna()
    #aca voy a pasar todos los datos al tipo numerico q corresponda
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int) #esta funcion me transforma todos los valores a numericos a int
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float) #esta funcion me transforma todos los valores a numericos a float
    # FINALIZA SANITIZACION



    #aca sucede la magia, vamos agregando cada mes al df ventas
    #a cada mes le cargo su dataset correspondiente, por eso uso la variable mes
    dataframes[mes] = df_ventas

dataframes['Enero']

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio
0,141234,iPhone,1,700.00,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"
...,...,...,...,...,...,...
9718,150497,20in Monitor,1,109.99,01/26/19 19:09,"95 8th St, Dallas, TX 75001"
9719,150498,27in FHD Monitor,1,149.99,01/10/19 22:58,"403 7th St, San Francisco, CA 94016"
9720,150499,ThinkPad Laptop,1,999.99,01/21/19 14:31,"214 Main St, Portland, OR 97035"
9721,150500,AAA Batteries (4-pack),2,2.99,01/15/19 14:21,"810 2nd St, Los Angeles, CA 90001"


In [ ]:
# Ordenar los meses

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
  dataframe_mes = dataframes[mes]

In [ ]:
# Obtener los datos de dia y hora de los pedidos

def extraer_datos(datos):
  df_ventas['Fecha de Pedido'] = pd.to_datetime(df_ventas['Fecha de Pedido'])
  datos['Hora'] = df_ventas['Fecha de Pedido'].dt.hour
  datos['Dia'] = df_ventas['Fecha de Pedido'].dt.day

dataframe_mes = extraer_datos(dataframe_mes)

In [ ]:
# Ordenar los meses
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

totales_ventas = []
nombres_meses = []

for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframes_mes = extraer_datos(dataframe_mes)

  #aca sumo el total de ventas de cada mes
  total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum() #aca obtengo el total de ventas de cada df(x mes)
  totales_ventas.append(total_ventas_mes)                   #append al array de la suma de total de ventas x cada mes(creo un array con las ventas de cada mes)
  #agregamos el nombre de cada mes, al array que contiene el nombre de todos los meses
  nombres_meses.append(mes)

In [ ]:
# Graficamos usando Plotly
import plotly.graph_objects as go

# Creo la figura

fig = go.Figure()

fig.add_trace(go.Bar(
    x=nombres_meses,
    y=totales_ventas,
    yaxis="y",
    name="Unidades vendidas",
    marker =dict(color=totales_ventas,colorscale= 'RdYlGn', colorbar= dict(title= "Total de ventas")),
    text = totales_ventas,
    textposition = 'auto'

))

fig.update_layout(
    title="Unidades vendidas por mes",
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side = "left")
)

fig.show()

In [ ]:
# Creo el objeto figura
Horas = list(range(24))
fig = go.Figure()


for mes in meses:
  dataframe_mes = dataframes[mes]
  dataframes_mes = extraer_datos(dataframe_mes)

  Ventas_por_hora= dataframe_mes.groupby('Hora')['Cantidad Pedida'].count()


  # Agrego al objeto figura un grafico de de linea con las horas y las ventas por hora
  fig.add_trace(go.Scatter(

      x=Horas,
      y=Ventas_por_hora,
      mode='lines',
      name=mes,

      line=dict(shape='spline', smoothing=1.3, width=2)))


  # Configuro los ejes y los títulos
fig.update_layout(

    title="Unidades vendidas por hora",
    xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left"))


fig.show()

In [ ]:
fig = go.Figure()

horas_analizadas = [12,19,14]

for hora in horas_analizadas:
  total_productos_vendidos = []

  for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframes_mes = extraer_datos(dataframe_mes)

    ventas_hora = dataframe_mes[dataframe_mes['Hora'] == hora]

    total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].sum())

  fig.add_trace(go.Scatter(

      x=meses,
      y=total_productos_vendidos,
      mode='lines',
      name=f'{hora} horas',
      line = dict(width= 2, shape= 'spline')
  ))


# Configuro los ejes y los títulos
fig.update_layout(

  title="Unidades vendidas por hora",
  xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
  yaxis=dict(title="Unidades vendidas", side="left"),
  legend=dict(y=0.3, yanchor='bottom')
)


fig.show()